# ショット切り出し  

以下の処理を行います。  

* ショット切り出し
* 中断区間のデータ除外
* 物理変換 + 校正
* SPM計算
* 事象記録のタグ付け
* SPMから算出される、ショット当たりの最大サンプル数を超えたショット除外
* Elasticsearchインデックスへの保存
  * shots-yyyyMMddHHMMSS-data：切り出されたショットデータ  
  * shots-yyyyMMddHHMMSS-meta：ショットのメタデータ
  

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from cut_out_shot.cut_out_shot import CutOutShot

### CutOutShot クラスパラメーター一覧  

**【必須】**  
***displacement_func***  
変位センサーの物理変換式  

***load01_func***  
荷重センサー01の物理変換式  

***load02_func***  
荷重センサー02の物理変換式  

***load03_func***  
荷重センサー03の物理変換式  

***load04_func***  
荷重センサー04の物理変換式  


**【任意】**  
***min_spm***  
最小SPM。この値をもとに、1ショットに含まれる最大サンプル数を計算し、最大サンプル数を超えたショットは切り出し対象外とする。  
指定しない場合、既定値の15となる。  

***back_seconds_for_tagging***  
データ収集マネージャーで記録した事象について、何秒遡ってタグ付けするかを指定。指定しない場合、既定値の120となる。

***previous_size***  
ショットを検知したサンプルから何サンプル遡ってショットに含めるかを指定。これは切り出しショットに荷重開始点が確実に含まれるようにするため。指定しない場合、既定値の1000となる。


In [2]:
# 物理変換式設定
displacement_func = lambda v: 135.0 - (v - 2.0) * 70.0 / 8.0
load01_func = lambda v: v * 100000 / 38000.0
load02_func = lambda v: v * 100000 / 38000.0
load03_func = lambda v: v * 100000 / 38000.0
load04_func = lambda v: v * 100000 / 38000.0

In [3]:
cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func
)

### cut_out_shot 関数パラメーター一覧
**【必須】**  
***rawdata_dir_name***  
ショット切り出しの対象とする生データファイルのディレクトリ（.pklファイルが含まれるディレクトリ）。  
ディレクトリ名はデータ収集開始日時で、yyyyMMddHHMMSSの形式となっている。

***start_displacement***  
ショット切り出しの開始を検知するための変位値。  

***end_displacement***  
ショット切り出しの終了を検知するための変位値。  

In [ ]:
start_displacement=47.0
end_displacement=34.0

cut_out_shot.cut_out_shot(
    rawdata_dir_name="20201201010000", 
    start_displacement=start_displacement, 
    end_displacement=end_displacement
)

[2021-01-27 12:58:31.255928] cut_out_shot start.
[2021-01-27 12:59:12.544761] Thoughput: 245124 doc/sec, processed_count: 10000000
[2021-01-27 12:59:34.866817] Thoughput: 316868 doc/sec, processed_count: 20000000
[2021-01-27 12:59:56.326319] Thoughput: 354705 doc/sec, processed_count: 30000000
[2021-01-27 13:00:18.005980] Thoughput: 376446 doc/sec, processed_count: 40000000
[2021-01-27 13:00:39.755820] Thoughput: 390604 doc/sec, processed_count: 50000000
[2021-01-27 13:01:01.040231] Thoughput: 401899 doc/sec, processed_count: 60000000
[2021-01-27 13:01:22.735954] Thoughput: 409388 doc/sec, processed_count: 70000000
[2021-01-27 13:01:44.037329] Thoughput: 416042 doc/sec, processed_count: 80000000
[2021-01-27 13:02:05.605339] Thoughput: 420843 doc/sec, processed_count: 90000000
[2021-01-27 13:02:26.998991] Thoughput: 425080 doc/sec, processed_count: 100000000
[2021-01-27 13:02:48.317934] Thoughput: 428734 doc/sec, processed_count: 110000000
[2021-01-27 13:03:09.858559] Thoughput: 431484 